In [120]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import requests, os
from zipfile import ZipFile
import shutil

Consts

In [177]:
ZIP_PATH = "http://people.csail.mit.edu/torralba/code/spatialenvelope/spatial_envelope_256x256_static_8outdoorcategories.zip"

Helper Functions

In [178]:
'''
Download and unzip file
'''
def load_files():
    filename = ZIP_PATH.split("/")[-1]
    with open(filename, "wb") as f:
        r = requests.get(ZIP_PATH)
        f.write(r.content)
    foldername = filename.split(".")[0]
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()
        nameList = zip_ref.namelist()
        imgs = [name.split("/")[-1] for name in nameList if name.endswith('.jpg')]
        forest_images = [name for name in imgs if name.startswith('/forest_')]
        mountain_images = [name for name in imgs if name.startswith('/mountain_')]
        print(forest_images)
    shutil.move(foldername, 'images')
    return forest_images,mountain_images


In [179]:
load_files()


[]


Error: Destination path 'images/spatial_envelope_256x256_static_8outdoorcategories' already exists

In [ ]:
def prepare_set(image_names):
    

SyntaxError: incomplete input (2754308233.py, line 2)

''''''
def prepare_set():
    

1. Compute dense SIFT from the images